In [2]:
from pathlib import Path

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import numbers

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier

import tensorflow as tf
from tensorflow import keras

In [3]:
PATH = Path.cwd()
DATA_PATH = Path.cwd() / 'data'
PREPROCESSED_DATA_PATH = DATA_PATH / 'preprocessed'
RESULTS_DATA_PATH = PATH / 'results'
SPLITTED_DATA_PATH = PREPROCESSED_DATA_PATH / 'splitted'

In [4]:
PREPROCESSED_KLEKOTA_ROTH_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP.csv'
PREPROCESSED_MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP.csv'
PREPROCESSED_EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_ExtFP.csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP.csv'
PREPROCESSED_KLEKOTA_ROTH__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-ExtFP.csv'
PREPROCESSED_MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP-ExtFP.csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv'

RESULTS_KLEKOTA_ROTH = RESULTS_DATA_PATH / 'KlekFP.csv'
RESULTS_MACCS = RESULTS_DATA_PATH / 'MACCSFP.csv'
RESULTS_EXT = RESULTS_DATA_PATH / 'ExtFP.csv'
RESULTS_KLEKOTA_ROTH__MACCS_DATA = RESULTS_DATA_PATH / 'KlekFP-MACCSFP.csv'
RESULTS_KLEKOTA_ROTH__EXT_DATA = RESULTS_DATA_PATH / 'KlekFP-ExtFP.csv'
RESULTS_MACCS__EXT_DATA = RESULTS_DATA_PATH / 'MACCSFP-ExtFP.csv'
RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA = RESULTS_DATA_PATH / 'KlekFP-MACCSFP-ExtFP.csv'

In [5]:
PREPROCESSED_KLEKOTA_ROTH_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP[{}].csv'
PREPROCESSED_MACCS_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_MACCSFP[{}].csv'
PREPROCESSED_EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_ExtFP[{}].csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP-MACCSFP[{}].csv'
PREPROCESSED_KLEKOTA_ROTH__EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP-ExtFP[{}].csv'
PREPROCESSED_MACCS__EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_MACCSFP-ExtFP[{}].csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv[{}]'

RESULTS_KLEKOTA_ROTH_X = RESULTS_DATA_PATH / '{}/KlekFP[{}].csv'
RESULTS_MACCS_X = RESULTS_DATA_PATH / '{}/MACCSFP[{}].csv'
RESULTS_EXT_X = RESULTS_DATA_PATH / '{}/ExtFP[{}].csv'
RESULTS_KLEKOTA_ROTH__MACCS_DATA_X = RESULTS_DATA_PATH / '{}/KlekFP-MACCSFP[{}].csv'
RESULTS_KLEKOTA_ROTH__EXT_DATA_X = RESULTS_DATA_PATH / '{}/KlekFP-ExtFP[{}].csv'
RESULTS_MACCS__EXT_DATA_X = RESULTS_DATA_PATH / '{}/MACCSFP-ExtFP[{}].csv'
RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA_X = RESULTS_DATA_PATH / '{}/KlekFP-MACCSFP-ExtFP[{}].csv'

In [5]:
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"Running GridSearchCV for {key} with params {self.params[key]} with number of features {len(X.columns)}")
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [6]:
class HyperparametersTuner():
    
    def apply(self, input_filepath, output_filepath):
        df = pd.read_csv(input_filepath)
    
        X = df.drop(['IC50','toxic'], axis=1)
        y = df['toxic']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
        
        print(f'Running Hyperparameters Tunning for {input_filepath}')
        helper = EstimatorSelectionHelper(self.__prepare_models(), self.__prepare_parameters())
        helper.fit(X_train, y_train, scoring='f1', n_jobs=4, verbose=2)
        
        score_summary = helper.score_summary()
        
        score_summary.to_csv(output_filepath, index = False)
        
        return score_summary
    
    @staticmethod
    def __prepare_models():
        return {
            'LogisticRegression': LogisticRegression(), 
            'RidgeClassifier': RidgeClassifier(), 
            'SGDClassifier': SGDClassifier(), 
            'SVC': SVC(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'RandomForestClassifier': RandomForestClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
        }
    
    @staticmethod
    def __prepare_parameters():
        return {
            'LogisticRegression': {
                'C': [0.01, 0.1, 1, 10],
                'penalty': ['l1', 'l2'],
                'max_iter': list(range(100,500,100)),
                'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
            },
            'RidgeClassifier': {
                'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            },
            'SGDClassifier': {
                'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                'penalty': ['l1', 'l2', 'elasticnet'],
                'max_iter': list(range(100,500,100)),
                'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
            },
            'SVC': {
                'C': [0.01, 0.1, 1, 10],
                'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
            },
            'ExtraTreesClassifier': {
                'n_estimators': list(range(10, 250, 50)),
                'criterion': ['gini', 'entropy'],
                'max_features': ['sqrt', 'log2']
            },
            'RandomForestClassifier': { 
                'n_estimators': list(range(10, 250, 50)),
            },
            'AdaBoostClassifier':  { 
                'n_estimators': list(range(10, 250, 50)),
            },
            'GradientBoostingClassifier': { 
                'n_estimators': list(range(10, 250, 50)),
                'learning_rate': [0.4, 0.6, 0.8, 1.0] 
            }
        }

In [19]:
data_to_process = [
    (PREPROCESSED_KLEKOTA_ROTH_DATA, RESULTS_KLEKOTA_ROTH), 
    (PREPROCESSED_MACCS_DATA, RESULTS_MACCS), 
    (PREPROCESSED_EXT_DATA, RESULTS_EXT),
    
    (PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA, RESULTS_KLEKOTA_ROTH__MACCS_DATA),
    (PREPROCESSED_KLEKOTA_ROTH__EXT_DATA, RESULTS_KLEKOTA_ROTH__EXT_DATA),
    (PREPROCESSED_MACCS__EXT_DATA, RESULTS_MACCS__EXT_DATA),
    (PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA, RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA)
]

hyperparameters_tuner = HyperparametersTuner()
results = []

for data in data_to_process:
    result = hyperparameters_tuner.apply(data[0], data[1])
    results.append(result)

Running GridSearchCV for LogisticRegression with params {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'max_iter': [100, 200, 300, 400], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
Fitting 3 folds for each of 160 candidates, totalling 480 fits


C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SG0306249\Min

Running GridSearchCV for RidgeClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Running GridSearchCV for SGDClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'penalty': ['l1', 'l2', 'elasticnet'], 'max_iter': [100, 200, 300, 400], 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']}
Fitting 3 folds for each of 600 candidates, totalling 1800 fits
Running GridSearchCV for SVC with params {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Running GridSearchCV for ExtraTreesClassifier with params {'n_estimators': [10, 60, 110, 160, 210], 'criterion': ['gini', 'entropy'], 'max_features': ['sqrt', 'log2']}
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Running GridSearchCV for RandomForestClassifier with params {'n_estimators': [10, 60, 110, 160

In [7]:
data_to_process = [
    (PREPROCESSED_KLEKOTA_ROTH_DATA_X, RESULTS_KLEKOTA_ROTH_X), 
    (PREPROCESSED_MACCS_DATA_X, RESULTS_MACCS_X), 
    (PREPROCESSED_EXT_DATA_X, RESULTS_EXT_X),
    
    (PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA_X, RESULTS_KLEKOTA_ROTH__MACCS_DATA_X),
    (PREPROCESSED_KLEKOTA_ROTH__EXT_DATA_X, RESULTS_KLEKOTA_ROTH__EXT_DATA_X),
    (PREPROCESSED_MACCS__EXT_DATA_X, RESULTS_MACCS__EXT_DATA_X),
    (PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA_X, RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA_X)
]

hyperparameters_tuner = HyperparametersTuner()
results = []

for x in range(500, 2500, 500):
    for data in data_to_process:
        input_path = Path(str(data[0]).format(x, x))
        output_path = Path(str(data[1]).format(x, x))
        try:
            result = hyperparameters_tuner.apply(input_path, output_path)
            results.append(result)
        except:
            print(f'Ignoring {input_path} because the file does not exists')

Running Hyperparameters Tunning for C:\Users\SG0306249\PycharmProjects\cardiotoxicity_prediction\data\preprocessed\splitted\2000\cardiotoxicity_hERG_KlekFP-MACCSFP[2000].csv
Running GridSearchCV for LogisticRegression with params {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'max_iter': [100, 200, 300, 400], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']} with number of features 2000
Fitting 3 folds for each of 160 candidates, totalling 480 fits


C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SG0306249\Min

Running GridSearchCV for RidgeClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]} with number of features 2000
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Running GridSearchCV for SGDClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'penalty': ['l1', 'l2', 'elasticnet'], 'max_iter': [100, 200, 300, 400], 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']} with number of features 2000
Fitting 3 folds for each of 600 candidates, totalling 1800 fits
Running GridSearchCV for SVC with params {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} with number of features 2000
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Running GridSearchCV for ExtraTreesClassifier with params {'n_estimators': [10, 60, 110, 160, 210], 'criterion': ['gini', 'entropy'], 'max_features': ['sqrt', 'log2']} with number of features 2000
Fitting 3 folds for each of 20 candidat

C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SG0306249\Min

Running GridSearchCV for RidgeClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]} with number of features 2000
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Running GridSearchCV for SGDClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'penalty': ['l1', 'l2', 'elasticnet'], 'max_iter': [100, 200, 300, 400], 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']} with number of features 2000
Fitting 3 folds for each of 600 candidates, totalling 1800 fits
Running GridSearchCV for SVC with params {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} with number of features 2000
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Running GridSearchCV for ExtraTreesClassifier with params {'n_estimators': [10, 60, 110, 160, 210], 'criterion': ['gini', 'entropy'], 'max_features': ['sqrt', 'log2']} with number of features 2000
Fitting 3 folds for each of 20 candidat

In [10]:
# PREPROCESSED_KLEKOTA_ROTH_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP.csv'
# PREPROCESSED_MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP.csv'
# PREPROCESSED_EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_ExtFP.csv'
# PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP.csv'
# PREPROCESSED_KLEKOTA_ROTH__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-ExtFP.csv'
# PREPROCESSED_MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP-ExtFP.csv'
# PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv'


df = pd.read_csv(PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA)

X = df.drop(['IC50','toxic'], axis=1)
y = df['toxic']
        
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
        
lol = len(X_train.columns)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test) 

# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(lol,)),
#     keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
#     keras.layers.Dense(1, activation=tf.nn.sigmoid),
# ])

import keras.layers as layers

# model = keras.Sequential([

#     # First Convolutional Block
#     layers.Conv1D(filters=32, kernel_size=5, activation="relu", padding='same', input_shape=(X_train,)),
#     layers.MaxPool1D(),

#     # Second Convolutional Block
#     layers.Conv1D(filters=64, kernel_size=3, activation="relu", padding='same'),
#     layers.MaxPool1D(),

#     # Third Convolutional Block
#     layers.Conv1D(filters=128, kernel_size=3, activation="relu", padding='same'),
#     layers.MaxPool1D(),

#     # Classifier Head
#     layers.Flatten(),
#     layers.Dense(units=6, activation="relu"),
#     layers.Dense(units=1, activation="sigmoid"),
# ])

# model = keras.Sequential([
#     keras.layers.Dense(128, input_shape=(len(X_train.columns),), kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
#     keras.layers.Dense(256, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
#     keras.layers.Dense(256, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
#     keras.layers.Dense(512, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
#     keras.layers.Dense(1, activation=tf.nn.sigmoid),
# ])

model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(lol,), kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
    keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

es = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                   mode='max',
                                   patience=20,
                                   restore_best_weights=True)

model.fit(X_train, y_train, 
          epochs=300, 
          batch_size=10,
          callbacks=[es],
          validation_split=0.2,
          shuffle=True)

test_loss, test_acc = model.evaluate(X_test, y_test)

print(test_loss, test_acc)

Epoch 1/300
681/681 [==============================] - 3s 4ms/step - loss: 3.7843 - accuracy: 0.6665 - val_loss: 1.7911 - val_accuracy: 0.6945
Epoch 2/300
681/681 [==============================] - 2s 2ms/step - loss: 1.4622 - accuracy: 0.6989 - val_loss: 1.3193 - val_accuracy: 0.6827
Epoch 3/300
681/681 [==============================] - 2s 3ms/step - loss: 1.2394 - accuracy: 0.7104 - val_loss: 1.1462 - val_accuracy: 0.7244
Epoch 4/300
681/681 [==============================] - 2s 2ms/step - loss: 1.1356 - accuracy: 0.7247 - val_loss: 1.1246 - val_accuracy: 0.7168
Epoch 5/300
681/681 [==============================] - 2s 3ms/step - loss: 1.0975 - accuracy: 0.7297 - val_loss: 1.1307 - val_accuracy: 0.7297
Epoch 6/300
681/681 [==============================] - 2s 2ms/step - loss: 1.1166 - accuracy: 0.7292 - val_loss: 1.0863 - val_accuracy: 0.7256
Epoch 7/300
681/681 [==============================] - 2s 2ms/step - loss: 1.0654 - accuracy: 0.7355 - val_loss: 1.0738 - val_accuracy: 0.7291